# Raspberry PI Assembly 1

## Sub Project 1: Control Structures 
-  Code up a C/C++ goto statement 
-  Code up a if then else statement, use the goto statement you just coded in the solution of the if then else
- Code up a x = (cond) ? thenbody :else body
- Code up a counted for statement using the if then and goto statements you have coded
- Code up a do ... while() condition

To complete the above you will need to come up with a way to generate labels and allocate variables to memory (and or) registers.


## Discussion of Solution to Sub Project 1
### C/C++ goto statement
### C/C++ do while statement
To implement a do while statement, that the $do$ is the start of the loop, and the $while$ is a conditional branch.

``` c
do 
{compound statement}
while(condition);
```

so an equivalent control structure which builds on the prior structures is
``` c
dolabelk:  
{compound statement}
if (condition) goto dolabelk;
```

To convert the following example:
``` c
int x=1;
do 
{printf('\n x is: %d',x}
while(++x<10);
```
to assembly the following steps need to be taken <br>

- allocate a variable to hold the integer x
- allocate a constant to hold the constant 10 (need to be careful with load immediate on ARM)
- create a label for the do
- figure out how to print out x (an advance concept, next project just use for now)
- get the integer
- increment x (this is because it is a post increment)
- compare to the constant
- goto the label if the condition is true 

In [1]:
!cat dowhile.s

.data
.balign 4
   string:  .asciz "\n x is : %d"
.balign 4
   x:       .word 0
.balign 4
   const1:  .word 10
/* CODE SECTION */
.text
.balign 4
.global main
.extern printf

main:
    push    {ip,lr}     @ This is needed to return to the Operating System

    ldr     r0,=x       @ x=1;
    mov     r1,#1
    str     r1,[r0]

do1:
    ldr     r0,=string  @ printf("\n x is : %d",x) Note: Printf may change registers R0,R1,R2,R3,R4
    ldr     r1,=x
    ldr     r1,[r1]
    bl      printf

    ldr     r0,=x
    ldr     r1,[r0]
    add     r1,r1,#1    @++X
    str     r1,[r0]
    ldr     r2,=const1
    ldr     r2,[r2]
    cmp     r1,r2       @ if (X<10)
    blt     do1

    mov     R0,#0

    pop     {ip, pc}    @ This is the return to the operating system


# Testing Design and Result
The code provided above should enter a while loop and print the numbers 1 through 9 inclusive, once the variable x is incremented to 10, it should exit.

In [2]:
!as -o dowhile.o dowhile.s
!gcc -o dowhile dowhile.o
!./dowhile


 x is : 1
 x is : 2
 x is : 3
 x is : 4
 x is : 5
 x is : 6
 x is : 7
 x is : 8
 x is : 9

## Sub Project 2: Non-Recursive Merge Sort

Using the following reference 
__[Merge Sort](https://stackoverflow.com/questions/1557894/non-recursive-merge-sort)__, implement the the non-recursive merge sort for powers of two. <br>
You will need to print out the sorted and unsorted lists.  You must use the control structures you coded up in sub project 1.

### Testing
- Test for lists of 16,32, and 128.
- provide a test case of length 16 (if any) which takes the fewest cycles
- provide a test case of length 16 (if any) which takes the most cycles


## discusion of Solution
Below is the code (downloaded on 9/12/2018 from __[Merge Sort](https://stackoverflow.com/questions/1557894/non-recursive-merge-sort)__)

``` c
float a[50000000],b[50000000];
void mergesort (long num)
{
    int rght, wid, rend;
    int i,j,m,t;

    for (int k=1; k < num; k *= 2 ) {       
        for (int left=0; left+k < num; left += k*2 ) {
            rght = left + k;        
            rend = rght + k;
            if (rend > num) rend = num; 
            m = left; i = left; j = rght; 
            while (i < rght && j < rend) { 
                if (a[i] <= a[j]) {         
                    b[m] = a[i]; i++;
                } else {
                    b[m] = a[j]; j++;
                }
                m++;
            }
            while (i < rght) { 
                b[m]=a[i]; 
                i++; m++;
            }
            while (j < rend) { 
                b[m]=a[j]; 
                j++; m++;
            }
            for (m=left; m < rend; m++) { 
                a[m] = b[m]; 
            }
        }
    }
}
```
The steps taken are:

- Verify the operation of the code in a c program
- implement the c code in assembly by hand translating each statement.

### Conversion Steps
To convert the code, we need to allocate variables, and provide code to initialize and print out the array to be sorted and the result.
#### Variable Allocation
The variables are identified and space allocated for each of the variables in the algothim

- m
- left
- rght
- rend
- j
- i
- k
- num 
- t 
- a : array to be sorted
- b : array used during the sorting

#### Array Definition, Initialization and Display
The next cell allocates a 128 element integer array, initializes it with random numbers and displays the result.


In [50]:
!cat arrays.s

.data
.balign 4
   string:  .asciz "\n A[%d] = : %d"
.balign 4
   A:       .skip 512  @ 128*4
.balign 4
   N:  .word 128

/* CODE SECTION */
.text
.balign 4
.global main
.extern printf
.extern rand

main:
    push    {ip,lr}     @ This is needed to return to the Operating System

    @@@  This bloc of code uses R4,R5,  R0,R1,R2,R3 are used for the call to random
    mov r5,#0           @ Initialize 128 elements in the array
    ldr r4,=A
loop1:
    ldr r0,=N
    ldr r0,[r0]
    cmp r5, r0
    bge end1
    bl      rand
    and r0,r0,#255
    str r0, [r4], #4
    add r5, r5, #1
    b loop1                  /* Go to the beginning of the loop */
end1:

    mov r5,#0           @ Print out the array
    ldr r4,=A
loop2:
    ldr r0,=N
    ldr r0,[r0]
    cmp r5, r0
    beq end2
    push {r0,r1,r2,r3,r4}  @ we can save and restore the registers on the stack!!
    ldr r0,=string
    mov r1,r5
    ldr r2,[r4],#4
    bl printf
    pop {r0,r1,r2,r3,r4}
    add r5, r5, #1

    b loop2              

In [3]:
!as -o arrays.o arrays.s
!gcc -o arrays arrays.o
!./arrays



 A[0] = : 103
 A[1] = : 198
 A[2] = : 105
 A[3] = : 115
 A[4] = : 81
 A[5] = : 255
 A[6] = : 74
 A[7] = : 236
 A[8] = : 41
 A[9] = : 205
 A[10] = : 186
 A[11] = : 171
 A[12] = : 242
 A[13] = : 251
 A[14] = : 227
 A[15] = : 70
 A[16] = : 124
 A[17] = : 194
 A[18] = : 84
 A[19] = : 248
 A[20] = : 27
 A[21] = : 232
 A[22] = : 231
 A[23] = : 141
 A[24] = : 118
 A[25] = : 90
 A[26] = : 46
 A[27] = : 99
 A[28] = : 51
 A[29] = : 159
 A[30] = : 201
 A[31] = : 154
 A[32] = : 102
 A[33] = : 50
 A[34] = : 13
 A[35] = : 183
 A[36] = : 49
 A[37] = : 88
 A[38] = : 163
 A[39] = : 90
 A[40] = : 37
 A[41] = : 93
 A[42] = : 5
 A[43] = : 23
 A[44] = : 88
 A[45] = : 233
 A[46] = : 94
 A[47] = : 212
 A[48] = : 171
 A[49] = : 178
 A[50] = : 205
 A[51] = : 198
 A[52] = : 155
 A[53] = : 180
 A[54] = : 84
 A[55] = : 17
 A[56] = : 14
 A[57] = : 130
 A[58] = : 116
 A[59] = : 65
 A[60] = : 33
 A[61] = : 61
 A[62] = : 220
 A[63] = : 135
 A[64] = : 112
 A[65] = : 233
 A[66] = : 62
 A[67] = : 161
 A[68] = : 65
 A[6

### The outer loop
The approach is to code and test the outer loops of the merge sort and verify the operation against the provided c code.
Then, after the outer loops, the inner merge step will be coded.
``` c
    int rght, wid, rend;
    int i,j,m,t;

    for (int k=1; k < num; k *= 2 ) {       
        for (int left=0; left+k < num; left += k*2 ) {
            rght = left + k;        
            rend = rght + k;
            if (rend > num) rend = num; 
            m = left; i = left; j = rght; 
        }
    }
``` 
The variables are:

- num : this is the variable N in the assemble .data section, we will put this in register R0
- k   : R1
- left : R2
- rght : R3
- rend : R4
- m    : R5
- i    : R6
- j    : R7

There are assembler diectives which you can use to make your code more readable, 
__[Assembler Directives](http://www.ic.unicamp.br/~celio/mc404-2014/docs/gnu-arm-directives.pdf)__



In [52]:
!cat Outerloop.s

.data
.balign 4
   string:  .asciz "\n A[%d] = : %d"
.balign 4
   string2: .asciz "\n k= %d,left=%d"
.balign 4
   A:       .skip 512 @128*4
.balign 4
   N:  .word 128

/* CODE SECTION */
.text
.balign 4
.global main
.extern printf
.extern rand

main:
    push    {ip,lr}     @ This is needed to return to the Operating System

    @@@  This bloc of code uses R4,R5,  R0,R1,R2,R3 are used for the call to random
    mov r5,#0           @ Initialize 128 elements in the array
    ldr r4,=A
loop1:
    ldr r0,=N
    ldr r0,[r0]
    cmp r5, r0
    bge end1
    bl      rand
    and r0,r0,#255
    str r0, [r4], #4
    add r5, r5, #1
    b loop1                  /* Go to the beginning of the loop */
end1:

    mov r5,#0           @ Print out the array
    ldr r4,=A
loop2:
    ldr r0,=N
    ldr r0,[r0]
    cmp r5, r0
    beq end2
    ldr r0,=string
    mov r1,r5
    ldr r2,[r4],#4
    bl printf
    add r5, r5, #1

    b loop2                  /* Go to the beginning of the loop */
end2:
@@@@@@@@@@@@@

In [4]:
!as -o Outerloop.o Outerloop.s
!gcc -o Outerloop Outerloop.o
!./Outerloop


 A[0] = : 103
 A[1] = : 198
 A[2] = : 105
 A[3] = : 115
 A[4] = : 81
 A[5] = : 255
 A[6] = : 74
 A[7] = : 236
 A[8] = : 41
 A[9] = : 205
 A[10] = : 186
 A[11] = : 171
 A[12] = : 242
 A[13] = : 251
 A[14] = : 227
 A[15] = : 70
 A[16] = : 124
 A[17] = : 194
 A[18] = : 84
 A[19] = : 248
 A[20] = : 27
 A[21] = : 232
 A[22] = : 231
 A[23] = : 141
 A[24] = : 118
 A[25] = : 90
 A[26] = : 46
 A[27] = : 99
 A[28] = : 51
 A[29] = : 159
 A[30] = : 201
 A[31] = : 154
 A[32] = : 102
 A[33] = : 50
 A[34] = : 13
 A[35] = : 183
 A[36] = : 49
 A[37] = : 88
 A[38] = : 163
 A[39] = : 90
 A[40] = : 37
 A[41] = : 93
 A[42] = : 5
 A[43] = : 23
 A[44] = : 88
 A[45] = : 233
 A[46] = : 94
 A[47] = : 212
 A[48] = : 171
 A[49] = : 178
 A[50] = : 205
 A[51] = : 198
 A[52] = : 155
 A[53] = : 180
 A[54] = : 84
 A[55] = : 17
 A[56] = : 14
 A[57] = : 130
 A[58] = : 116
 A[59] = : 65
 A[60] = : 33
 A[61] = : 61
 A[62] = : 220
 A[63] = : 135
 A[64] = : 112
 A[65] = : 233
 A[66] = : 62
 A[67] = : 161
 A[68] = : 65
 A[6

### Next step
Once the results are compared against the C code the next step is to code the swaps (the inner body of the loops)

``` c
            rght = left + k;        
            rend = rght + k;
            if (rend > num) rend = num; 
            m = left; i = left; j = rght; 
            while (i < rght && j < rend) { 
                if (a[i] <= a[j]) {         
                    b[m] = a[i]; i++;
                } else {
                    b[m] = a[j]; j++;
                }
                m++;
            }
            while (i < rght) { 
                b[m]=a[i]; 
                i++; m++;
            }
            while (j < rend) { 
                b[m]=a[j]; 
                j++; m++;
            }
            for (m=left; m < rend; m++) { 
                a[m] = b[m]; 
            }
```

## Sub Project 3: Recursive Merge Sort
Using the mege sort on this page __[Merge Sort](https://www.geeksforgeeks.org/merge-sort/)__
, and the control structures coded up in sub project 1 repeat 
the testing of sub problem 2 for the recursive merge sort. (download from reference on 9/20/2018)

``` c
// Merges two subarrays of arr[]. 
// First subarray is arr[l..m] 
// Second subarray is arr[m+1..r] 
void merge(int arr[], int l, int m, int r) 
{ 
    int i, j, k; 
    int n1 = m - l + 1; 
    int n2 =  r - m; 
  
    /* create temp arrays */
    int L[n1], R[n2]; 
  
    /* Copy data to temp arrays L[] and R[] */
    for (i = 0; i < n1; i++) 
        L[i] = arr[l + i]; 
    for (j = 0; j < n2; j++) 
        R[j] = arr[m + 1+ j]; 
  
    /* Merge the temp arrays back into arr[l..r]*/
    i = 0; // Initial index of first subarray 
    j = 0; // Initial index of second subarray 
    k = l; // Initial index of merged subarray 
    while (i < n1 && j < n2) 
    { 
        if (L[i] <= R[j]) 
        { 
            arr[k] = L[i]; 
            i++; 
        } 
        else
        { 
            arr[k] = R[j]; 
            j++; 
        } 
        k++; 
    } 
  
    /* Copy the remaining elements of L[], if there 
       are any */
    while (i < n1) 
    { 
        arr[k] = L[i]; 
        i++; 
        k++; 
    } 
  
    /* Copy the remaining elements of R[], if there 
       are any */
    while (j < n2) 
    { 
        arr[k] = R[j]; 
        j++; 
        k++; 
    } 
} 
  
/* l is for left index and r is right index of the 
   sub-array of arr to be sorted */
void mergeSort(int arr[], int l, int r) 
{ 
    if (l < r) 
    { 
        // Same as (l+r)/2, but avoids overflow for 
        // large l and h 
        int m = l+(r-l)/2; 
  
        // Sort first and second halves 
        mergeSort(arr, l, m); 
        mergeSort(arr, m+1, r); 
  
        merge(arr, l, m, r); 
    } 
} 
```
## Discussion of solution.
First, we need to be able to call a subroutine.  The $bl\ label$ instruction places the current value of the program counter ($PC$) into the linkage register ($LR$) before branching to the labels address.  In RTL this is;
- $LR \leftarrow PC, PC \leftarrow addessof(label) $

In the subroutine we can save any registers we may change, and the link register, and then at the end of the routine restore the registers and place the restored link register into the program counter.  The steps are:
- Save the link register and the registers we will use: push {R0,R1,R2,...,LR}
- subroutine code 
- Restore the registers: pop {R0,R1,R2,....,PC}  Notice: $PC \leftarrow LR$

Alternatively there is a $BX\ REG$ which can be used to place the restored $LR$ into the program counter $PC \leftrightarrow LR$.
### Approach
The idea is to code up the recursive part first, and print out all of the variables, then after this is verified we can code up the merge (non-recursive) part of the sort and verify operation.  So we need to allocate variables to locations, in this case placing them in a registers seems to make sense since it is easy to save them onto the stack.

Looking at the function call: 
```c
mergeSort(int arr[], int l, int r) 
```
we see the formal parameters, so:
- R0 = address or arr[] (this is a reference to the array)
- R1 = l, the left index into the subarray
- R2 = r, the right index into the subarray

Below is the code which is to be converted.
```c
mergeSort(int arr[], int l, int r) 
    { 
    if (l < r) 
    { 
        // Same as (l+r)/2, but avoids overflow for 
        // large l and h 
        int m = l+(r-l)/2; 
  
        // Sort first and second halves 
        mergeSort(arr, l, m); 
        mergeSort(arr, m+1, r); 
  
      // Not doing the merge step for now
      //  merge(arr, l, m, r); 
    } 
} 
```
To convert to assembly:
```c
@mergeSort(int arr[], int l, int r) 
mergeSort: push {r0,r1,r2,LR}
@    { 
@    if (l < r) 
     cmp r1,r2
     bge mergeSortEnd
@    { 
@        // Same as (l+r)/2, but avoids overflow for 
@        // large l and h 
         push {r1,r2}   @ save L and R
@        int m = l+(r-l)/2; 
         sub r2,r2,r1  # This was a bug (was a 1) int the morning and afternoon versions
         lsr r2,r2,#1
         add r2,r2,r1
         push {r2}  @ save m
@        // Sort first and second halves 
@        mergeSort(arr, l, m);   // Remember l is R1, R2 is r=m   
         bl mergeSort
         pop {r3}   @ put m into r3 
         pop  {r1,r2}  @restored l and r
         push {r1,r2}
         push {r3}
         add  r1,r3,#1
@        mergeSort(arr, m+1, r); 
         bl mergeSort
         pop {r3}
         pop  {r1,r2}
@      // Not doing the merge step for now
@      //  merge(arr, l, m, r); 
@   >>>>>    Note the formal params are r0=arr,r1=l,r3=m,r2=r <<<<<<<<<<<<<<<<<
@    } 
mergeSortEnd: pop {r0,r1,r2,PC}  @ notice put LR into PC to force return
@} 
```

## Testing Plan or Design
The idea is to use the C code to generate output of the left and right indices to verify the operation of the assembly code. The code below 
``` c
#include <stdio.h>

int test[128];

void mergeSort(int arr[], int l, int r)
{
    printf("\n %d,%d",l,r);
    if (l < r)
    {
        // Same as (l+r)/2, but avoids overflow for
        // large l and h
        int m = l+(r-l)/2;

        // Sort first and second halves
        mergeSort(arr, l, m);
        mergeSort(arr, m+1, r);

        //merge(arr, l, m, r);
    }
}

int main()
{
    mergeSort(test,0,128);
}
```

In [1]:
!gcc -o recurse recurse.c
!./recurse


 0,128
 0,64
 0,32
 0,16
 0,8
 0,4
 0,2
 0,1
 0,0
 1,1
 2,2
 3,4
 3,3
 4,4
 5,8
 5,6
 5,5
 6,6
 7,8
 7,7
 8,8
 9,16
 9,12
 9,10
 9,9
 10,10
 11,12
 11,11
 12,12
 13,16
 13,14
 13,13
 14,14
 15,16
 15,15
 16,16
 17,32
 17,24
 17,20
 17,18
 17,17
 18,18
 19,20
 19,19
 20,20
 21,24
 21,22
 21,21
 22,22
 23,24
 23,23
 24,24
 25,32
 25,28
 25,26
 25,25
 26,26
 27,28
 27,27
 28,28
 29,32
 29,30
 29,29
 30,30
 31,32
 31,31
 32,32
 33,64
 33,48
 33,40
 33,36
 33,34
 33,33
 34,34
 35,36
 35,35
 36,36
 37,40
 37,38
 37,37
 38,38
 39,40
 39,39
 40,40
 41,48
 41,44
 41,42
 41,41
 42,42
 43,44
 43,43
 44,44
 45,48
 45,46
 45,45
 46,46
 47,48
 47,47
 48,48
 49,64
 49,56
 49,52
 49,50
 49,49
 50,50
 51,52
 51,51
 52,52
 53,56
 53,54
 53,53
 54,54
 55,56
 55,55
 56,56
 57,64
 57,60
 57,58
 57,57
 58,58
 59,60
 59,59
 60,60
 61,64
 61,62
 61,61
 62,62
 63,64
 63,63
 64,64
 65,128
 65,96
 65,80
 65,72
 65,68
 65,66
 65,65
 66,66
 67,68
 67,67
 68,68
 69,72
 69,70
 69,69
 70,70
 71,72
 71,71
 72,72
 73,

## References
__[ARM Assembly on Raspberry PI](https://thinkingeek.com/arm-assembler-raspberry-pi/)__ <br>
__[ARM Assembly CodeBlocks](http://www.microdigitaled.com/ARM/ARM_ASM_books.htm)__<br>